In [1]:
import requests
import pandas as pd

In [2]:
# Verify if API is ON

result_on = requests.get('http://localhost:5000/')
result_on.text

'{"message":"API available","status":"success"}\n'

In [3]:
#Load insurance dataset
df = pd.read_csv('heart.csv')

In [4]:
# Analysing columns
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [6]:
# PREPARE Endpoint
df_json = df.to_json()
post_data = {'max_na': 0.2, 'max_var': 0.2, 'payload': df_json}
url = 'http://localhost:5000/prepare'

result_prepare = requests.post(url, data=post_data)
df_prep = pd.read_json(result_prepare.text)


In [7]:
df_prep.head()

,age,sex,cp,trestbps,fbs,restecg,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,1,0,0,2.3,0,0,1,1
1,37,1,2,130,0,1,0,3.5,0,0,2,1
2,41,0,1,130,0,0,0,1.4,2,0,2,1
3,56,1,1,120,0,1,0,0.8,2,0,2,1
4,57,0,0,120,0,1,1,0.6,2,0,2,1


In [8]:
df_prep.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
fbs           int64
restecg       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [9]:
# TRAIN_TEST Endpoint
url = 'http://localhost:5000/train_test'
ml_method = 'Classification'
train_columns = ''
target = 'target'
df_json = df_prep.to_json()
post_data = {'train_columns': ','.join(train_columns), 'target_column': target, 'ml_method': ml_method, 'test_size': 0.2, 'payload': df_json}
result_train_test = requests.post(url, data=post_data)
print(result_train_test.text)

{"KNN":{"Accuracy Score":0.6229508196721312},"gradient":{"Accuracy Score":0.8852459016393442},"randon_forest":{"Accuracy Score":0.8524590163934426}}



In [10]:
#TRAIN Endpoint
url = 'http://localhost:5000/train_model'
post_data = {'train_columns': ','.join(train_columns), 'target_column': target, 'ml_method': ml_method, 'model_name': 'KNN', 'payload': df_json}
result_train = requests.post(url, data=post_data)
print(result_train.text)

{"file_name":"models_registry/2021-12-29-19-51-54/KNN_classifier.pickle","file_name_param":"models_registry/2021-12-29-19-51-54/KNN_classifier.json"}



In [12]:
#PREDICT Endpoint
df_predict = df_prep.drop('target', axis=1)
df_json = df_predict.to_json()
model_path = result_train.text.split(',')[0].split(':')[1].replace('"','').replace(' ','')
url = 'http://localhost:5000/predict'
post_data = {'model_path': model_path, 'payload': df_json}

result_predict = requests.post(url, data=post_data)
print(result_predict.text)


{"prediction":{"0":0,"1":1,"2":1,"3":1,"4":1,"5":1,"6":1,"7":1,"8":0,"9":1,"10":1,"11":1,"12":1,"13":0,"14":1,"15":1,"16":1,"17":1,"18":0,"19":0,"20":1,"21":1,"22":1,"23":1,"24":1,"25":1,"26":1,"27":1,"28":1,"29":1,"30":1,"31":0,"32":1,"33":0,"34":1,"35":1,"36":1,"37":1,"38":1,"39":1,"40":1,"41":1,"42":1,"43":1,"44":1,"45":1,"46":1,"47":1,"48":1,"49":1,"50":1,"51":1,"52":0,"53":0,"54":0,"55":1,"56":1,"57":1,"58":1,"59":1,"60":1,"61":1,"62":1,"63":1,"64":1,"65":1,"66":1,"67":1,"68":1,"69":0,"70":1,"71":1,"72":1,"73":1,"74":1,"75":1,"76":1,"77":1,"78":1,"79":1,"80":1,"81":1,"82":1,"83":1,"84":1,"85":1,"86":1,"87":1,"88":1,"89":1,"90":1,"91":1,"92":1,"93":1,"94":0,"95":1,"96":0,"97":1,"98":1,"99":1,"100":0,"101":0,"102":1,"103":1,"104":1,"105":1,"106":0,"107":1,"108":1,"109":0,"110":0,"111":1,"112":1,"113":0,"114":1,"115":1,"116":1,"117":1,"118":1,"119":1,"120":0,"121":1,"122":1,"123":1,"124":1,"125":1,"126":0,"127":1,"128":1,"129":1,"130":0,"131":1,"132":1,"133":1,"134":1,"135":1,"136":1